In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#import netCDF4
#from scipy import stats
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
import sys
sys.path.append('/Users/gbromley/code/python_utilities/')
from python_data_functions import extract_months
#import shapely as sp

In [3]:
import geopandas as gp
from shapely.geometry import Point
#import ogr, gdal
import fiona

ImportError: No module named 'fiona'

In [4]:
data_dir = '/Users/gbromley/data/CRU/tmp/'
temp_file = 'cru_ts3.24.1901.2015.tmp.dat.nc'

nc_temp = xr.open_dataset(data_dir+temp_file)

In [5]:
t2m = nc_temp['tmp']

In [6]:
t2m

<xarray.DataArray 'tmp' (time: 1380, lat: 360, lon: 720)>
[357696000 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 -177.8 -177.2 -176.8 ...
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 -87.75 -87.25 -86.75 ...
  * time     (time) datetime64[ns] 1901-01-16 1901-02-15 1901-03-16 ...
Attributes:
    long_name: near-surface temperature
    units: degrees Celsius
    correlation_decay_distance: 1200.0

In [ ]:
df = pd.DataFrame(t2m['lon'].values,t2m['lat'].values)

In [ ]:
t_season='MJJ'
t_months=[5,6,7]
start_year = '1970-01-01'
end_year = '2010-12-01'
ext_e = -90
ext_w = -120
ext_n = 55
ext_s = 30


In [7]:
 test = gp.read_file('/Users/gbromley/Dropbox/EPSCOR-TrackII/Data/GIS_Boundaries/WAFER_Boundary.shp')

In [12]:
test.crs

{'init': u'epsg:26713'}

In [ ]:
test_t2m = t2m.isel(time=0).to_pandas()

In [ ]:
test_t2m = pd.DataFrame(test_t2m)

In [ ]:
test_t2m

In [ ]:
test_t2m['geometry'] = test_t2m.apply(lambda x: Point((float(t2m['lon'].values), float(t2m['lat'].values))), axis=1)


In [ ]:
t2m['lat'].values

In [ ]:
res_intersection = gp.overlay(test, test_t2m, how='intersection')


In [2]:
from rasterio import features
from affine import Affine

def transform_from_latlon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(shapes, coords, fill=np.nan, **kwargs):
    """Rasterize a list of (geometry, fill_value) tuples onto the given
    xray coordinates. This only works for 1d latitude and longitude
    arrays.
    """
    transform = transform_from_latlon(coords['latitude'], coords['longitude'])
    out_shape = (len(coords['latitude']), len(coords['longitude']))
    raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    return xray.DataArray(raster, coords=coords, dims=('latitude', 'longitude'))


In [ ]:
#extract the time period we are interested in
t2m_slice=t2m.sel(time=slice(start_year,end_year))
#grab the months interested in
t2m_months = t2m_slice.sel(time=extract_months(t2m_slice['time.month'],t_months[0],t_months[-1]))
#create month averages
t2m_months_avg = t2m_months.groupby('time.year').mean(dim='time')

#extract the 30 yr climate normal times
t2m_clim_30= t2m.sel(time=slice('1981-01-01','2010-12-01'))
#grab the months
t2m_clim_months_30 = t2m_clim_30.sel(time=extract_months(t2m_clim_30['time.month'],t_months[0],t_months[-1]))
#create averages over months for climate normal
t2m_clim_months_avg = t2m_clim_months_30.mean(dim='time')

In [ ]:
#create 2d structure to hold slope
spatial_trend = t2m.isel(time=1).copy(deep=True)
spatial_trend.name = 't2m_trend'
spatial_trend.attrs['units'] = 'C per Decade'
spatial_trend.attrs['long_name'] = '2 meter temperature trend'

#create 2d structure to hold pvalues
pvalues= t2m.isel(time=1).copy(deep=True)
pvalues.name = 't2m_trend_pvalues'
pvalues.attrs['units'] = 'pvalues'
pvalues.attrs['long_name'] = '2 meter temperature trend pvalues'

In [ ]:
#calculate slope and pvalue for each grid point
#replace with a map() call?
for i in np.arange(0,len(t2m['lat'])):
    for j in np.arange(0,len(t2m['lon'])):
        series = t2m_months_avg[:,i,j]
        anom = series-t2m_clim_months_avg[i,j]
        slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(0,len(anom)),anom)
        spatial_trend[i,j]=slope*10
        pvalues[i,j]=p_value
        #print(i)

In [ ]:
spatial_trend.to_dataset().to_netcdf(data_dir+'CRU_tmp_trends_'+t_season+'_'+'1970'+'.nc')
pvalues.to_dataset().to_netcdf(data_dir+'CRU_tmp_trends_pvalue'+t_season+'_'+'1970'+'.nc')


In [ ]:
fig, ax = plt.subplots();
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_longitude=-111.0,central_latitude=46.0,false_easting=0.0, false_northing=0.0, standard_parallels=(40.0, 50.0), globe=None))
ax.set_global()
test = spatial_trend.plot.pcolormesh(axes = ax, transform=ccrs.PlateCarree(),vmin=-1, vmax=1, cmap='RdBu_r')
#ax = plt.axes(projection=ccrs.LambertConformal())

ax.set_extent([ext_e, ext_w, ext_s, ext_n])
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')
ax.add_feature(states_provinces, edgecolor='gray')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
plt.title('CRU Temperature Trend 1970-2010 '+t_season,fontsize=24)
test.colorbar.set_label('$^\circ$C / Decade',fontsize=18)
test.colorbar.ax.tick_params(labelsize=14)
fig.set_size_inches(13,7)
fig.set_dpi=1200.0
#plt.savefig('CRU_temp_'+t_season+'_1970.png',bbox_inches='tight')
plt.show()

In [ ]:
anom_series = t2m_months - t2m_clim_months_avg
extent = anom_series.sel(lat=np.arange(ext_s,ext_n,step=.5))
extent = extent.sel(lon=np.arange(ext_w+360,ext_e+360,step=.5))
extent_series = extent.mean(dim=['lat','lon'])
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(0,len(extent_series)),extent_series.values)
line = slope*np.arange(0,len(extent_series.values))+intercept



fig, ax = plt.subplots();
plt.scatter(extent_series['year'],extent_series.values, c=extent_series.values,vmin=-3,vmax=2,cmap='RdBu_r',s=30);
#trend = ax.plot(anom['time'],line)
base = np.zeros(len(extent_series.values))
ax.plot(extent_series['year'],base,c='grey')
ax.set_title('North America 2m Temperature Anomaly '+t_season+' 1970-2010')
ax.set_xlabel('Year')
ax.set_ylabel('Temp Anomaly [C]')
trend = ax.plot(extent_series['year'],line)
#fig.savefig('2m_temp_anom_'+t_season+'_na_sp_1970.png')



In [ ]:
anom_series